In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import gzip
import pandas as pd


In [2]:
def get_col_names(filepath, col_names_line_starting_symbol):
    # ie: "#Symbol" for "data/clinvar/2023_01/hgvs4variation.txt.gz"

    with gzip.open(filepath, "rt") as f:
        for line_no, line in enumerate(f):
            # print(line)
            if line.startswith(col_names_line_starting_symbol):
                line = line[1:] # skipping the # symbol
                line = line.rstrip()
                col_names = [x for x in line.split('\t')]
                # if line_no==100: break
                break
        return col_names

get_col_names(home_dir+"data/clinvar/hgvs4variation_08022023.txt.gz", "#Symbol")

['Symbol',
 'GeneID',
 'VariationID',
 'AlleleID',
 'Type',
 'Assembly',
 'NucleotideExpression',
 'NucleotideChange',
 'ProteinExpression',
 'ProteinChange',
 'UsedForNaming',
 'Submitted',
 'OnRefSeqGene']

In [4]:
hgvs_data_filepath = home_dir+"data/clinvar/hgvs4variation_08022023.txt.gz"
hgvs_col_names = get_col_names(hgvs_data_filepath, "#Symbol")
hgvs_df = pd.read_csv(hgvs_data_filepath, compression='gzip', comment='#', delim_whitespace=False, sep="\t", header=None, names=hgvs_col_names)    
print(hgvs_df.shape)

/tmp/ipykernel_578462/3354852199.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  hgvs_df = pd.read_csv(hgvs_data_filepath, compression='gzip', comment='#', delim_whitespace=False, sep="\t", header=None, names=hgvs_col_names)


(17217771, 13)


In [5]:
clinvar_data_filepath = home_dir+"data/clinvar/search_results/clinvar_HumanPathogenicMissenseVariants01012022To14022023.txt"
clinvar_search_result_df = pd.read_csv(clinvar_data_filepath, delim_whitespace=False, sep="\t")
print(clinvar_search_result_df.shape)
print(clinvar_search_result_df.columns)

(4006, 16)
Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)',
       'Clinical significance (Last reviewed)', 'Review status', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Unnamed: 15'],
      dtype='object')


In [6]:
clinvar_marged_with_prots_df = clinvar_search_result_df.merge(hgvs_df, how="inner", left_on="VariationID", right_on="VariationID")
print(clinvar_marged_with_prots_df.shape)
# clinvar_marged_with_prots_df.head()

(28297, 28)


In [7]:
clinvar_marged_with_prots_df = clinvar_marged_with_prots_df[clinvar_marged_with_prots_df.ProteinExpression!="-"] # removing variants isoforms that is not associated with any proteins
print(clinvar_marged_with_prots_df.shape)
clinvar_marged_with_prots_df = clinvar_marged_with_prots_df[clinvar_marged_with_prots_df.ProteinExpression.str.contains("NP_")] # removing variants isoforms that is not associated with any protein accession id
print(clinvar_marged_with_prots_df.shape)
clinvar_marged_with_prots_df.reset_index(inplace=True, drop=True)
clinvar_marged_with_prots_df.head()

(14492, 28)
(13558, 28)


,Name,Gene(s),Protein change,Condition(s),Clinical significance (Last reviewed),Review status,Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,...,AlleleID,Type,Assembly,NucleotideExpression,NucleotideChange,ProteinExpression,ProteinChange,UsedForNaming,Submitted,OnRefSeqGene
0,NM_001394713.1(PERM1):c.2330T>C (p.Val777Ala),PERM1,"V663A, V755A, V777A",Renal tubular epithelial cell apoptosis|Neutro...,Pathogenic,no assertion criteria provided,VCV001320032,1,911595,1,...,1310278,coding,na,NM_001291366.2:c.2330T>C,c.2330T>C,NP_001278295.1:p.Val777Ala,p.Val777Ala,No,No,No
1,NM_001394713.1(PERM1):c.2330T>C (p.Val777Ala),PERM1,"V663A, V755A, V777A",Renal tubular epithelial cell apoptosis|Neutro...,Pathogenic,no assertion criteria provided,VCV001320032,1,911595,1,...,1310278,coding,na,NM_001291367.2:c.1988T>C,c.1988T>C,NP_001278296.2:p.Val663Ala,p.Val663Ala,Yes,No,No
2,NM_001394713.1(PERM1):c.2330T>C (p.Val777Ala),PERM1,"V663A, V755A, V777A",Renal tubular epithelial cell apoptosis|Neutro...,Pathogenic,no assertion criteria provided,VCV001320032,1,911595,1,...,1310278,coding,na,NM_001369897.1:c.2330T>C,c.2330T>C,NP_001356826.1:p.Val777Ala,p.Val777Ala,Yes,No,No
3,NM_001394713.1(PERM1):c.2330T>C (p.Val777Ala),PERM1,"V663A, V755A, V777A",Renal tubular epithelial cell apoptosis|Neutro...,Pathogenic,no assertion criteria provided,VCV001320032,1,911595,1,...,1310278,coding,na,NM_001369898.1:c.2264T>C,c.2264T>C,NP_001356827.1:p.Val755Ala,p.Val755Ala,Yes,No,No
4,NM_001394713.1(PERM1):c.2330T>C (p.Val777Ala),PERM1,"V663A, V755A, V777A",Renal tubular epithelial cell apoptosis|Neutro...,Pathogenic,no assertion criteria provided,VCV001320032,1,911595,1,...,1310278,coding,na,NM_001394713.1:c.2330T>C,c.2330T>C,NP_001381642.1:p.Val777Ala,p.Val777Ala,Yes,No,No


In [8]:
clinvar_pathogenic_isoforms_df = clinvar_marged_with_prots_df
print(clinvar_pathogenic_isoforms_df.shape)
print(clinvar_pathogenic_isoforms_df["Gene(s)"].unique().shape)
print(clinvar_pathogenic_isoforms_df["GRCh38Chromosome"].unique()) # no Y-chr found
clinvar_pathogenic_isoforms_df.to_csv(home_dir+"data/clinvar/filtered/clinvar_PathogenicIsoforms01012022To14022023.txt", sep="\t", index=False)

(13558, 28)
(1208,)
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' 'X']


In [9]:
mane_select_transcription_mask = clinvar_marged_with_prots_df["Name"].apply(lambda x: x.split(":")[0].split("(")[0]) == clinvar_marged_with_prots_df["NucleotideExpression"].apply(lambda x: x.split(":")[0])
mane_transcripted_df = clinvar_marged_with_prots_df[mane_select_transcription_mask]
mane_transcripted_df.reset_index(drop=True, inplace=True)
# mane_transcripted_df.head()
mane_transcripted_df.to_csv(home_dir+"data/clinvar/filtered/clinvar_MANETranscriptedPathogenicMissenseVariants01012022To14022023.txt", sep="\t", index=False)

In [10]:
print(mane_transcripted_df.shape)
print(mane_transcripted_df["Gene(s)"].unique().shape)
print(mane_transcripted_df["GRCh38Chromosome"].unique()) # no Y-chr found

(3097, 28)
(1202,)
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' 'X']


In [52]:
import time
import os
from Bio import Entrez, SeqIO
from urllib.error import HTTPError

Entrez.email = "akabir0101@gmail.com"
Entrez.api_key = "328570309ccd040632796143ec88b51bcf08"
retmax = 500

protein_acc_list = mane_transcripted_df.ProteinExpression.apply(lambda x: x.split(":")[0]).to_list()
print(f"#-of proteins to download: {len(protein_acc_list)}")
    
for i, prot_acc in enumerate(protein_acc_list):
    print(i, prot_acc)
    out_filepath = home_dir+f"data/clinvar/fastas/{prot_acc}.fasta"

    if i>=10: break
    if os.path.exists(out_filepath): continue

    attempt = 0
    while (attempt < 3):
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="protein", id=prot_acc, rettype="fasta", retmode="text")
            # print(handle.read())
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(10)
            else:
                raise
    if fetch_handle:
        f_out_handle = open(out_filepath, 'w')
        seq_record = SeqIO.read(fetch_handle, "fasta")
        # print(record)
        # print(record.id)
        # print(str(record.seq))
        SeqIO.write(seq_record, f_out_handle, "fasta")
        
        fetch_handle.close()
        f_out_handle.close()
    

#-of proteins to download: 8416
0 NP_001381642.1
1 NP_940978.2
2 NP_542172.2
3 NP_542172.2
4 NP_114127.3
5 NP_001164006.1
6 NP_001164006.1
7 NP_001164006.1
8 NP_001164006.1
9 NP_002065.1
10 NP_002065.1
